In [1]:
import pandas as pd

# Load full dataset (may take ~30 sec)
df = pd.read_csv("../data/raw/accepted_2007_to_2018Q4.csv", low_memory=False)

# Filter to only 2018 issues
df_2018 = df[df['issue_d'].str.contains("2018", na=False)]

# Check shape and save trimmed version
print("✅ Trimmed shape:", df_2018.shape)
df_2018.to_csv("../data/raw/accepted_2018.csv", index=False)

✅ Trimmed shape: (495242, 151)
